In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import networkx as nx

In [2]:
# Command to use the full widht of the screen
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }/style>"))

In [3]:
def BALCL(init_nodes_x, final_nodes_x, init_nodes_y, final_nodes_y, m_param_x, m_param_y, l_param=0, CN=False, Initial_Connectivity='all-to-all'):
    assert init_nodes_x > 1
    assert init_nodes_y > 1
    assert init_nodes_x < final_nodes_x
    assert init_nodes_y < final_nodes_y
    assert init_nodes_x >= m_param_x
    assert init_nodes_y >= m_param_y
    assert l_param >= 0

    #INITIAL NETWORK:
    A = np.zeros((final_nodes_x, final_nodes_y))
    if Initial_Connectivity == 'all-to-all':
        for i in range(init_nodes_x):
            for j in range(init_nodes_y):
                A[i,j] = 1
    elif Initial_Connectivity == 'my_random':
        for i in range(init_nodes_x):
            for j in range(init_nodes_y):
                A[i,j] = int(np.random.random()+0.5)
            if not A[i].any() != 0:
                #print("Empty row:", i, A[i])
                rand_column = np.random.choice(range(init_nodes_y))
                A[i, rand_column] = 1
        for j in range(init_nodes_y):
            if not A[:,j].any() != 0:
                #print("Empty column:", j, A[:,j])
                rand_row = np.random.choice(range(init_nodes_x))
                A[rand_row, j] = 1
    print("Initial A")
    print(A)
    
    #NETWORK EVOLUTION:
    X = np.arange(init_nodes_x, final_nodes_x)
    Y = np.arange(init_nodes_y, final_nodes_y)
    x = len(X)
    y = len(Y)
    if x >= y:
        c = x/y
        for i in range(y):
            for j in range(int(c)):
                idx = j+i*int(c)
                p_deg_target = np.sum(A[:,:Y[i]], axis=0)
                p_deg_target /= np.sum(p_deg_target)
                rand_target = np.random.choice(Y[i], size=m_param_x, p=p_deg_target, replace=False)
                A[X[idx], rand_target] = 1
                print("\nSecond A")
                print(A)
            p_deg_source = np.sum(A[:X[idx]+1,:], axis=1)
            p_deg_source /= np.sum(p_deg_source)
            rand_source = np.random.choice(X[idx]+1, size=m_param_y, p=p_deg_source, replace=False)
            A[rand_source, Y[i]] = 1
            print("Third A")
            print(A)
        for k in range(idx+1, len(X)):
            p_deg_target = np.sum(A[:,:Y[i]], axis=0)
            p_deg_target /= np.sum(p_deg_target)
            rand_target = np.random.choice(Y[i], size=m_param_x, p=p_deg_target, replace=False)
            A[X[k], rand_target] = 1
            print("Fourth A")
            print(A)
    else:
        c = y/x
        for i in range(x):
            for j in range(int(c)):
                idx = j+i*int(c)
                p_deg_source = np.sum(A[:X[i],:], axis=1)
                p_deg_source /= np.sum(p_deg_source)
                rand_source = np.random.choice(X[i], size=m_param_y, p=p_deg_source, replace=False)
                A[rand_source, Y[idx]] = 1
                print("\nSecond A")
                print(A)
            p_deg_target = np.sum(A[:,:Y[idx]+1], axis=0)
            p_deg_target /= np.sum(p_deg_target)
            rand_target = np.random.choice(Y[idx]+1, size=m_param_x, p=p_deg_target, replace=False)
            A[X[i], rand_target] = 1
            print("Third A")
            print(A)
        for k in range(idx+1, len(Y)):
            p_deg_source = np.sum(A[:X[i],:], axis=1)
            p_deg_source /= np.sum(p_deg_source)
            rand_source = np.random.choice(X[i], size=m_param_y, p=p_deg_source, replace=False)
            A[rand_source, Y[k]] = 1
            print("Fourth A")
            print(A)
            
    return A

In [4]:
testBALCL = BALCL(4, 9, 2, 5, 1, 2, 0, CN=True, Initial_Connectivity='all-to-all')
testBALCL

Initial A
[[1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

Second A
[[1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Third A
[[1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

Second A
[[1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Third A
[[1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

Second A
[[1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0

array([[1., 1., 0., 0., 0.],
       [1., 1., 1., 1., 1.],
       [1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])